# New OSM

To run this notebook, an Isochrones API key is required from https://openrouteservice.org/dev/#/home?tab=1


In [142]:
import requests
import xml.etree.ElementTree as ET
import folium
import webbrowser
import time
import osmnx as ox
import geopandas as gpd
import openrouteservice.isochrones as ors
from os import path
from openrouteservice import client


# my API key:
api_key = '5b3ce3597851110001cf6248e7fe8693c29a490d84a78e28d41c16a1'

melb = 'City of Melbourne, Australia'
melbourne = ox.geocode_to_gdf(melb)

In [143]:
# Melbourne city bounding box
bbox = [-37.850667, 144.896981,-37.775451, 144.991351]

url = f'https://overpass-api.de/api/interpreter?' \
      f'data=[out:json];' \
      f'(node["shop"="supermarket"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["shop"="supermarket"]({",".join([str(x) for x in bbox])}););out;' \
      f'(node["amenity"="school"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["amenity"="school"]({",".join([str(x) for x in bbox])}););out;' \
      f'(node["amenity"="hospital"]({",".join([str(x) for x in bbox])}););out;' \
      f'(way["amenity"="hospital"]({",".join([str(x) for x in bbox])}););out;' \
      f'>;' \
      f'out skel qt;'

r = requests.get(url)
data = r.json()

In [144]:
# Create a dict to store all supermarkets of interest extracted from OSM
data_subset = {}

for element in data['elements']:

    # Check that the element has a 'tags' dict and that the dict, if it exists, has the 'name' listed
    if 'tags' in element and 'name' in element['tags']:

        # If the element is a way, ie comprised of multiple nodes, then take the first node as the location
        if element['type'] == 'way':

            # for supermarkets, which are shops:
            if 'shop' in element['tags']:
                
                data_subset[element['id']] = {
                'amenity': element['tags']['shop']
                , 'display name': element['tags']['name']
                , 'type': 'way'
                , 'location': element['nodes'][0]
            }
            
            # for hospitals and schools, which are amenities:
            elif 'amenity' in element['tags']:
                
                data_subset[element['id']] = {
                'amenity': element['tags']['amenity']
                , 'display name': element['tags']['name']
                , 'type': 'way'
                , 'location': element['nodes'][0]
            }

            # If the element is a single node, store the location of the node
        elif element['type'] == 'node':
            
            # for supermarkets, which are shops:
            if 'shop' in element['tags']:
                
                data_subset[element['id']] = {
                'amenity': element['tags']['shop']
                , 'display name': element['tags']['name']
                , 'type': 'node'
                , 'location': [element['lat'], element['lon']]
            }
                
            elif 'amenity' in element['tags']:
                
                data_subset[element['id']] = {
                'amenity': element['tags']['amenity']
                , 'display name': element['tags']['name']
                , 'type': 'node'
                , 'location': [element['lat'], element['lon']]
            }

        else:
            print(f"Location {element['tags']['name']} does not appear to be readable")
                

In [145]:
# Extract the 'ways' from all supermarkets
way_nodes = [data_subset[key]['location'] for key in data_subset.keys() if data_subset[key]['type'] == 'way']

way_locations = []

# Query the API in batches of 100 nodes for speed.
for i in range(0, len(way_nodes), 100):

    # Query the batch of nodes
    url = f"https://overpass-api.de/api/interpreter?" \
          f"node(id:{','.join(map(str, way_nodes[i:100 * (i + 1)]))});" \
          f"out;"
    r = requests.get(url)

    # XML tree output
    tree = ET.fromstring(r.text)
    way_locations.extend([x.attrib for x in tree.iter()][3:])

In [146]:
# Get the OSM IDs of each element in the API response
way_keys = [x['id'] for x in way_locations]

# Assign the API response coordinates to the data
for key in data_subset.keys():
    if data_subset[key]['type'] == 'way':

        idx = way_keys.index(str(data_subset[key]['location']))
        data_subset[key]['location'] = [way_locations[idx]['lat'], way_locations[idx]['lon']]

In [147]:
# Generate the API parameters
params = {
    # 'profile': 'driving-car'
    'profile': 'foot-walking'
    # 'profile': 'cycling-regular'
    , 'range_type': 'distance'
    , 'range': [400]  # 400 metre and 800 m isochrones based on 
    , 'attributes': ['total_pop']
}

# Set this to true if not hosting your own Openrouteservice instance
throttle_requests = True

In [198]:
# # Cooment after running, query can only be done once with the free key

# # The API key is stored in the secrets.py file
# ors = client.Client(key=api_key)

# # Loop over the stations and query the API for the isochrone data, then add each station to the map
# for key in data_subset.keys():
    
#     # sleep every three seconds to ensure query parameters are not exceeded (limit 20 per minute)
#     if throttle_requests:
#         time.sleep(3)

#     # Add supermarket coordinates to params and query ORS.
#     params['locations'] = [list(reversed(data_subset[key]['location']))]
#     data_subset[key]['isochrone'] = ors.isochrones(**params)



In [194]:
m = folium.Map(
    tiles='cartodb positron'                                            # Monochromatic map
    # tiles='openstreetmap'                                         # Normal map
    , location=(-37.8136, 144.9631)   # Centre the map using the bounding box coords
    , zoom_start=13
)

# Create a dictionary mapping each supermarket chain to its respective brand colours.
# Note that we're using lowercase supermarket names as the keys.
amenities = ['hospital', 'school', 'supermarket']
colours = ['#BA3C19', '#3171AE', '#2C7329']

colour_dict = dict(zip([amenity.lower() for amenity in amenities], colours))

In [195]:
# Layer Groups

sch_layer = folium.FeatureGroup(name='Schools')
sch_layer.add_to(m)

sup_layer = folium.FeatureGroup(name='Supermarkets')
sup_layer.add_to(m)

for key in data_subset.keys():
    
    if 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'supermarket':
    # Add isochrone to the map
        folium.features.GeoJson(
            data=data_subset[key]['isochrone']
            , style_function=lambda feature: dict(color='#2C7329')
        ).add_to(sup_layer)
    
    elif 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'school':
    # Add isochrone to the map
        folium.features.GeoJson(
            data=data_subset[key]['isochrone']
            , style_function=lambda feature: dict(color='#3171AE')
        ).add_to(sch_layer)
    
    else:
        continue
# m.save('Amenities_melbourne.html')
# webbrowser.open(path.realpath('Amenities_melbourne.html'), new=2)

In [196]:

for key in data_subset.keys():    
    
    if 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'school':
    
        # Add a school marker to map:
        folium.map.Marker(
            data_subset[key]['location'],
            icon=folium.Icon(color='white',
                             icon_color='#3171AE',
                             icon='graduation-cap',
                             prefix='fa'
                            ),
            popup=f"{data_subset[key]['display name']}"
                f" Population: {int(data_subset[key]['isochrone']['features'][0]['properties']['total_pop'])}"
        ).add_to(sch_layer)
    
    # # hospital marker
    # elif 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'hospital':
    #     hospital = folium.features.CustomIcon('Icons/hospital.png', icon_size=(20, 20))
    #     folium.map.Marker(
    #         data_subset[key]['location'],
    #         icon=hospital,
    #         popup=f"{data_subset[key]['display name']}"
    #         f", Population: {int(data_subset[key]['isochrone']['features'][0]['properties']['total_pop'])}"
    #     ).add_to(m)
    
    
    elif 'isochrone' in data_subset[key] and data_subset[key]['amenity'] == 'supermarket':
        
        # supermarket marker
        folium.map.Marker(
            data_subset[key]['location'],
            icon=folium.Icon(color='white',
                             icon_color='#2C7329',
                             icon='shopping-cart',
                             prefix='fa'
                            ),
            popup=f"{data_subset[key]['display name']}"
                f" Population: {int(data_subset[key]['isochrone']['features'][0]['properties']['total_pop'])}"
        ).add_to(sup_layer)
        

In [197]:
folium.LayerControl(position='topright', collapsed=True).add_to(m)
m

In [ ]:
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm

# function to produce a map of greenspace and their values

def greenspace_map(gdf, latitude, longitude, col_name):
    try:
        m = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='cartodb positron')
        gdf.explore(gdf[col_name], 
                    cmap=['#5c6b28','#7c9d45','#a47e4f'], 
                    tooltip=col_name, 
                    m=m,
                    popup=f"{gdf['park_name']}"
                f" Average NDVI value: {gdf['median_NDVI']}",
                    categorical=True)
        
        return m
    except Exception as e:
        return str(e)